In [1]:
import pandas as pd
import requests as req
import sqlite3
from bs4 import BeautifulSoup
from datetime import datetime

In [2]:
log_file = "./log_file.txt"
url = "https://web.archive.org/web/20230908091635%20/https://en.wikipedia.org/wiki/List_of_largest_banks"
table_attrib = ["Name","MC_USD_Billion"]
csv_path = "C:\\Users\\ravik\\Documents\\IBM Engineering\\Python\\Banking_Project\\exhange_rate.csv"
outputfile = "Largest_banks_data.csv"
Database_Name = 'Banks.db'
Table_Name = 'Largest_banks'
sql_connection = sqlite3.connect(Database_Name)
query1 = 'SELECT * FROM Largest_banks'
query2 = 'SELECT AVG(MC_GBP_Billion) FROM Largest_banks'
query3 = 'SELECT Name from Largest_banks LIMIT 5'

In [3]:
def log_progress(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S'
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open(log_file,"a") as f:
       f.write(timestamp + ' : ' + message + '\n')

In [4]:
def extract(url,table_attrib):
    response = req.get(url).text
    Data = BeautifulSoup(response,"html.parser")
    Tables = Data.find_all("table",{"class":"wikitable"})
    rows = Tables[0].find_all('tr')

    extracted_data = []

    for row in rows:
        col = row.find_all('td')
        if len(col)!=0:
            data_dic = {
                "Name": col[1].text.strip(),
                "MC_USD_Billion":float(col[2].contents[0][:-1])
            }
            extracted_data.append(data_dic)
    df = pd.DataFrame(extracted_data,columns=table_attrib)
    print(df)
    return df

In [5]:
def transform(df,csv_path):
    exchangerate_df = pd.read_csv(csv_path)
    exchangerate_dict = exchangerate_df.set_index('Currency').to_dict()['Rate']

    df['MC_EUR_Billion'] = ((df['MC_USD_Billion'].astype(float))*exchangerate_dict['EUR']).round(2)
    df['MC_GBP_Billion'] = ((df['MC_USD_Billion'].astype(float))*exchangerate_dict['GBP']).round(2)
    df['MC_INR_Billion'] = ((df['MC_USD_Billion'].astype(float))*exchangerate_dict['INR']).round(2)

    return df

In [6]:
def load_to_csv(df, outputfile):
    df.to_csv(outputfile)

In [7]:
def load_to_db(df,sql_connection,Table_Name):
    df.to_sql(Table_Name,sql_connection,if_exists='replace',index=False)

In [8]:
def run_query(queryStatement, sql_connection):
    print(queryStatement)
    query_output = pd.read_sql(queryStatement, sql_connection)
    print(query_output)

In [10]:
log_progress('Extaction has started')
df = extract(url,table_attrib)
log_progress('Extaction has Ended')
log_progress('Tranformation has started')
transform(df,csv_path)
log_progress('Tranformation has Ended')
log_progress('Loading to csv has started')
load_to_csv(df,outputfile)
log_progress('Loading to csv has ended')
log_progress('Loading to db has started')
load_to_db(df,sql_connection,Table_Name)
log_progress('Loading to db has ended')
log_progress(f'running query: {query1}')
run_query(query1,sql_connection)
log_progress(f'running query: {query2}')
run_query(query2,sql_connection)
log_progress(f'running query: {query3}')
run_query(query3,sql_connection)

                                      Name  MC_USD_Billion
0                           JPMorgan Chase          432.92
1                          Bank of America          231.52
2  Industrial and Commercial Bank of China          194.56
3               Agricultural Bank of China          160.68
4                                HDFC Bank          157.91
5                              Wells Fargo          155.87
6                        HSBC Holdings PLC          148.90
7                           Morgan Stanley          140.83
8                  China Construction Bank          139.82
9                            Bank of China          136.81
SELECT * FROM Largest_banks
                                      Name  MC_USD_Billion  MC_EUR_Billion  \
0                           JPMorgan Chase          432.92          402.62   
1                          Bank of America          231.52          215.31   
2  Industrial and Commercial Bank of China          194.56          180.94   
3          